In [2]:
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_squared_error, accuracy_score
import os
import statistics
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)

Using TensorFlow backend.


In [3]:
benign_datas = pd.read_pickle('/home/sdsra/Downloads/CTU-13-Dataset/benign/datas-final-by-origin-tool.pkl')

In [4]:
benign_datas['is_malware'] = 0

In [5]:
malware_datas = pd.read_pickle('/home/sdsra/Downloads/CTU-13-Dataset/malware/1/datas-by-origin-tool.pkl')
for i in [2,3,4,5,6,7,8,9,10,11,12,13,42,43,44,45,46,47,48,49,50,52,53,54]:
    pkl_path = '/home/sdsra/Downloads/CTU-13-Dataset/malware/{}/datas-by-origin-tool.pkl'.format(str(i))
    if os.path.exists(pkl_path):
        tmp_data = pd.read_pickle(pkl_path)
        malware_datas = pd.concat([malware_datas, tmp_data], ignore_index=True)

In [6]:
malware_datas = malware_datas[malware_datas['label'] == 'botnet']

In [7]:
malware_datas['is_malware'] = 1

In [8]:
len(benign_datas), len(malware_datas)

(6778, 599)

In [129]:
# malware_datas = malware_datas[malware_datas['label'] == 'botnet']
upsampled_malware_datas = malware_datas.sample(len(benign_datas), replace=True)

In [130]:
upsampled_malware_datas['is_malware'] = 1

In [131]:
datas = pd.concat([benign_datas, upsampled_malware_datas], ignore_index=True)

/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [10]:
datas = pd.concat([benign_datas, malware_datas], ignore_index=True)

/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [11]:
numeric_columns = list(filter(lambda x: x not in ['src_ip','src_port','dst_ip','dst_port','proto'], benign_datas.columns))

In [12]:
datas['is_malware'].value_counts()

0    6778
1     599
Name: is_malware, dtype: int64

In [13]:
X_train, X_test, y_train, y_test = train_test_split(datas, datas['is_malware'], test_size=0.33, random_state=42)

## Fit regression

In [14]:
params_regressor = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf_regressor = GradientBoostingRegressor(**params_regressor)
score = cross_validate(clf_regressor, datas[numeric_columns], datas['is_malware'], cv=10, return_train_score=False)

In [15]:
score

{'fit_time': array([0.63262558, 0.58617997, 0.58340454, 0.58375382, 0.58422494,
        0.58254457, 0.58103871, 0.58826756, 0.59512806, 0.08647323]),
 'score_time': array([0.00170088, 0.00165701, 0.0016489 , 0.0016911 , 0.00166202,
        0.00166655, 0.00166583, 0.00164771, 0.00166416, 0.00139308]),
 'test_score': array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , -4.34057971])}

In [16]:
params_regressor = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf_regressor = GradientBoostingRegressor(**params_regressor)

In [17]:
clf_regressor.fit(X_train[numeric_columns], y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [18]:
mse_regressor = mean_squared_error(y_test, clf_regressor.predict(X_test[numeric_columns]))

In [19]:
mse_regressor

3.2398515915950465e-06

In [20]:
X_test['predict_score'] = clf_regressor.predict(X_test[numeric_columns])

/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
X_test[['is_malware', 'predict_score']]

,is_malware,predict_score
1103,0,0.000532
2338,0,0.000532
7197,1,0.993961
4013,0,0.000532
2684,0,0.000532
2846,0,0.000532
2705,0,0.000532
2522,0,0.000532
3989,0,0.000532
2825,0,0.000532


## classifier

In [22]:
# cross validation
params_classifier = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'exponential'}
clf_classifier = GradientBoostingClassifier(**params_classifier)
score = cross_validate(clf_classifier, datas[numeric_columns], datas['is_malware'], cv=10, return_train_score=False)

In [23]:
score

{'fit_time': array([0.87833786, 0.8267138 , 0.83342767, 0.83306599, 0.83505511,
        0.83577394, 0.82919931, 0.83021832, 0.83439064, 0.83882165]),
 'score_time': array([0.00171328, 0.00168157, 0.0016644 , 0.00169158, 0.00168395,
        0.00172806, 0.00169396, 0.00173211, 0.00167918, 0.00165772]),
 'test_score': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [24]:
params_classifier = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'exponential'}
clf_classifier = GradientBoostingClassifier(**params_classifier)

In [25]:
clf_classifier.fit(X_train[numeric_columns], y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='exponential', max_depth=4,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [26]:
score = accuracy_score(y_test, clf_classifier.predict(X_test[numeric_columns]))

In [27]:
score

1.0

In [28]:
X_test['is_malware_predict'] = clf_classifier.predict(X_test[numeric_columns])

/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
X_test[['is_malware', 'is_malware_predict']]

,is_malware,is_malware_predict
1103,0,0
2338,0,0
7197,1,1
4013,0,0
2684,0,0
2846,0,0
2705,0,0
2522,0,0
3989,0,0
2825,0,0
